In [1]:
import trax

INFO:tensorflow:tokens_length=568 inputs_length=512 targets_length=114 noise_density=0.15 mean_noise_span_length=3.0 


In [18]:
from trax import layers as tl
import os

In [3]:
train_stream = trax.data.TFDS('imdb_reviews', keys=('text','label'), train=True)()

Shuffling and writing examples to /home/prhyme/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete48N9DN/imdb_reviews-train.tfrecord


Shuffling and writing examples to /home/prhyme/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete48N9DN/imdb_reviews-test.tfrecord


Shuffling and writing examples to /home/prhyme/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete48N9DN/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /home/prhyme/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
eval_stream = trax.data.TFDS('imdb_reviews', keys=('text','label'), train=False)()

In [5]:
data_pipeline = trax.data.Serial(
    trax.data.Tokenize(vocab_file='en_8k.subword',keys=[0]),
    trax.data.Shuffle(),
    trax.data.FilterByLength(max_length=2048, length_keys=[0]),
    trax.data.BucketByLength(boundaries=[32, 128, 512, 2048],
                            batch_sizes=[512,128,32,8,1],
                            length_keys=[0]),
    trax.data.AddLossWeights()
)

In [6]:
train_batches_stream = data_pipeline(train_stream)

In [7]:
eval_batches_stream = data_pipeline(eval_stream)

In [8]:
example_batch = next(train_batches_stream)

In [9]:
print(f'shapes={[x.shape for x in example_batch]}')

shapes=[(8, 2048), (8,), (8,)]


In [10]:
model = tl.Serial(
    tl.Embedding(vocab_size=8192, d_feature=256),
    tl.Mean(axis=1),
    tl.Dense(2),
    tl.LogSoftmax()
)

In [13]:
from trax.supervised import training

In [14]:
train_task = training.TrainTask(
    labeled_data=train_batches_stream,
    loss_layer=tl.CrossEntropyLoss(),
    optimizer=trax.optimizers.Adam(0.01),
    n_steps_per_checkpoint=500
)

/home/prhyme/.local/lib/python3.8/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [15]:
eval_task = training.EvalTask(
    labeled_data=eval_batches_stream,
    metrics = [tl.CrossEntropyLoss(), tl.Accuracy()],
    n_eval_batches=20
)

In [16]:
# Training loop saves checkpoints to output_dir

In [19]:
output_dir = os.path.expanduser('~/output_dir/')

In [20]:
!rm -rf {output_dir}

In [21]:
training_loop = training.Loop(model,
                             train_task,
                             eval_tasks=[eval_task],
                             output_dir=output_dir)

In [22]:
training_loop.run(2000)


Step      1: Ran 1 train steps in 2.08 secs
Step      1: train CrossEntropyLoss |  0.69912976
Step      1: eval  CrossEntropyLoss |  0.70418198
Step      1: eval          Accuracy |  0.46406250

Step    500: Ran 499 train steps in 17.23 secs
Step    500: train CrossEntropyLoss |  0.49487442
Step    500: eval  CrossEntropyLoss |  0.40184160
Step    500: eval          Accuracy |  0.84375000

Step   1000: Ran 500 train steps in 14.92 secs
Step   1000: train CrossEntropyLoss |  0.35941410
Step   1000: eval  CrossEntropyLoss |  0.47291508
Step   1000: eval          Accuracy |  0.78281250

Step   1500: Ran 500 train steps in 16.19 secs
Step   1500: train CrossEntropyLoss |  0.33607489
Step   1500: eval  CrossEntropyLoss |  0.30258615
Step   1500: eval          Accuracy |  0.87343750

Step   2000: Ran 500 train steps in 15.87 secs
Step   2000: train CrossEntropyLoss |  0.28492057
Step   2000: eval  CrossEntropyLoss |  0.41767745
Step   2000: eval          Accuracy |  0.80625000


In [23]:
example_input = next(eval_batches_stream)[0][0]

In [25]:
example_input_str = trax.data.detokenize(example_input, vocab_file='en_8k.subword')

In [26]:
example_input_str

'One of Chaplin\'s longest films up to that point, Burlesque on Carmen is a clever and surprisingly complex parody of what was then "Prosper Merimee\'s" well-known story about "Carmen." I was a little confused about the difference between the IMDb\'s listing of the 1915 Burlesque on Carmen and the 1916 version. Based on the running time I assume that it was the 1915 version that I saw, since the 1916 one is a good 20 minutes longer, and from what I\'ve read, those are 20 unnecessary and unimpressive minutes. <br /><br />From the very beginning, it\'s clear that Burlesque on Carmen is one of Chaplin\'s most complex and ambitious efforts to date, starting off with a long back story, told through inter-titles, about the tragic love story of Carmen. <br /><br />Carmen is sent by a band of gypsies ("A band who put the GYP in gypsy."), to seduce a Spanish officer so they can pull off their smuggling operation. It\'s a clever, Chaplinesque band of criminals, the leader of whom, Lillas Pastia,

In [27]:
sentiment_log_probs = model(example_input[None,:1])

In [29]:
import numpy as np

In [31]:
np.exp(sentiment_log_probs)

array([[1.7103437e-36, 1.0000000e+00]], dtype=float32)